# Imports

In [29]:
import importlib
from functools import partial

from torch import nn
import torch
import pandas as pd

import decision_learning.modeling.pipeline
import decision_learning.data.shortest_path_grid

from decision_learning.modeling.models import LinearRegression
from decision_learning.modeling.pipeline import lossfn_experiment_pipeline, lossfn_hyperparam_grid
from decision_learning.data.shortest_path_grid import genData

In [10]:
import decision_learning.modeling.train
importlib.reload(decision_learning.modeling.train)

importlib.reload(decision_learning.modeling.pipeline)
from decision_learning.modeling.pipeline import lossfn_experiment_pipeline, lossfn_hyperparam_grid 

import decision_learning.data.shortest_path_grid
importlib.reload(decision_learning.data.shortest_path_grid)
from decision_learning.data.shortest_path_grid import genData

import decision_learning.modeling.loss
importlib.reload(decision_learning.modeling.loss)
import decision_learning.modeling.loss

# Example Setup

### Optimization Model

# Tell people what the optimization model output should look like

In [3]:
def shortest_path_solver(costs, size, sens = 1e-4):
    # Forward Pass
    starting_ind = 0
    starting_ind_c = 0
    samples = costs.shape[0]
    V_arr = torch.zeros(samples, size ** 2)
    for i in range(0, 2 * (size - 1)):
        num_nodes = min(i + 1, 9 - i)
        num_nodes_next = min(i + 2, 9 - i - 1)
        num_arcs = 2 * (max(num_nodes, num_nodes_next) - 1)
        V_1 = V_arr[:, starting_ind:starting_ind + num_nodes]
        layer_costs = costs[:, starting_ind_c:starting_ind_c + num_arcs]
        l_costs = layer_costs[:, 0::2]
        r_costs = layer_costs[:, 1::2]
        next_V_val_l = torch.ones(samples, num_nodes_next) * float('inf')
        next_V_val_r = torch.ones(samples, num_nodes_next) * float('inf')
        if num_nodes_next > num_nodes:
            next_V_val_l[:, :num_nodes_next - 1] = V_1 + l_costs
            next_V_val_r[:, 1:num_nodes_next] = V_1 + r_costs
        else:
            next_V_val_l = V_1[:, :num_nodes_next] + l_costs
            next_V_val_r = V_1[:, 1:num_nodes_next + 1] + r_costs
        next_V_val = torch.minimum(next_V_val_l, next_V_val_r)
        V_arr[:, starting_ind + num_nodes:starting_ind + num_nodes + num_nodes_next] = next_V_val

        starting_ind += num_nodes
        starting_ind_c += num_arcs

    # Backward Pass
    starting_ind = size ** 2
    starting_ind_c = costs.shape[1]
    prev_act = torch.ones(samples, 1)
    sol = torch.zeros(costs.shape)
    for i in range(2 * (size - 1), 0, -1):
        num_nodes = min(i + 1, 9 - i)
        num_nodes_next = min(i, 9 - i + 1)
        V_1 = V_arr[:, starting_ind - num_nodes:starting_ind]
        V_2 = V_arr[:, starting_ind - num_nodes - num_nodes_next:starting_ind - num_nodes]

        num_arcs = 2 * (max(num_nodes, num_nodes_next) - 1)
        layer_costs = costs[:, starting_ind_c - num_arcs: starting_ind_c]

        if num_nodes < num_nodes_next:
            l_cs_res = ((V_2[:, :num_nodes_next - 1] - V_1 + layer_costs[:, ::2]) < sens) * prev_act
            r_cs_res = ((V_2[:, 1:num_nodes_next] - V_1 + layer_costs[:, 1::2]) < sens) * prev_act
            prev_act = torch.zeros(V_2.shape)
            prev_act[:, :num_nodes_next - 1] += l_cs_res
            prev_act[:, 1:num_nodes_next] += r_cs_res
        else:
            l_cs_res = ((V_2 - V_1[:, :num_nodes - 1] + layer_costs[:, ::2]) < sens) * prev_act[:, :num_nodes - 1]
            r_cs_res = ((V_2 - V_1[:, 1:num_nodes] + layer_costs[:, 1::2]) < sens) * prev_act[:, 1:num_nodes]
            prev_act = torch.zeros(V_2.shape)
            prev_act += l_cs_res
            prev_act += r_cs_res
        cs = torch.zeros(layer_costs.shape)
        cs[:, ::2] = l_cs_res
        cs[:, 1::2] = r_cs_res
        sol[:, starting_ind_c - num_arcs: starting_ind_c] = cs

        starting_ind = starting_ind - num_nodes
        starting_ind_c = starting_ind_c - num_arcs
    # Dimension (samples, num edges)
    obj = torch.sum(sol * costs, axis=1)
    # Dimension (samples, 1)
    return sol.to(torch.float32), obj.reshape(-1,1).to(torch.float32)

### Data Generation Setup

In [4]:
torch.manual_seed(105)
indices_arr = torch.randperm(100000)
indices_arr_test = torch.randperm(100000)

sim = 0
n_arr = [200, 400, 800, 1600]
ep_arr = ['unif', 'normal']
trials = 100

exp_arr = []
for n in n_arr:
    for ep in ep_arr:
        for t in range(trials):
            exp_arr.append([n, ep, t])

# setup
exp = exp_arr[0]
ep_type = exp[1]
trial = exp[2]

# generate data
grid = (5, 5)  # grid size
num_data = exp[0]  # number of training data
num_feat = 5  # size of feature
deg = 6  # polynomial degree
e = .3  # noise width

# path planting for shortest path example
planted_good_pwl_params = {'slope0':0, 
                    'int0':2,
                    'slope1':0, 
                    'int1':2}
planted_bad_pwl_params = {'slope0':4, 
                    'int0':0,
                    'slope1':0, 
                    'int1':2.2}

plant_edge = True

print(num_data, ep_type, trial)

200 unif 0


# Testing Pipeline
Necessary components
- data (features, true costs): train-test splits
- prediction model
- optimization model
- existing loss functions (hyperparameter configs)
- custom loss functions
- misc params

In [5]:
importlib.reload(decision_learning.modeling.pipeline)
from decision_learning.modeling.pipeline import lossfn_experiment_pipeline, lossfn_hyperparam_grid 

importlib.reload(decision_learning.modeling.train)
from decision_learning.modeling.train import train

In [6]:
# ------------DATA------------
# training data
generated_data = genData(num_data=num_data+200,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
# testing data
generated_data_test = genData(num_data=10000,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr_test[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)

# ------------prediction model------------
pred_model = LinearRegression(input_dim=generated_data['feat'].shape[1],
                 output_dim=generated_data['cost'].shape[1])

# ------------optimization model------------
optmodel = partial(shortest_path_solver,size=5)

# ------------custom loss function------------
custom_loss_inputs = [{'loss_name':'cosine',
                      'loss':nn.CosineEmbeddingLoss,
                      'data': {'X': generated_data['feat'],
                               'input2':generated_data['cost'], 
                               'target':torch.ones(generated_data['cost'].shape[0])}
                      }
                     ]


2024-11-26 19:06:32,342 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-26 19:06:32,342 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-26 19:06:32,343 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-26 19:06:32,343 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-26 19:06:32,348 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-26 19:06:32,348 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-26 19:06:32,418 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 

### Get SPO, MSE, Cosine First

In [17]:
result_metrics, trained_models = lossfn_experiment_pipeline(X_train=generated_data['feat'],
                true_cost_train=generated_data['cost'],
                X_test=generated_data_test['feat'],
                true_cost_test=generated_data_test['cost_true'], 
                predmodel=pred_model,
                optmodel=optmodel,
                val_split_params={'test_size':200, 'random_state':42},
                loss_names=['SPO+', 'MSE'],                
                custom_loss_inputs=custom_loss_inputs,
                training_configs={'num_epochs':100,
                                  'lr': 0.01,
                                 'dataloader_params': {'batch_size':200, 'shuffle':True}},
                save_models=True
                )

/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/train.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = {key: torch.tensor(value, dtype=torch.float32) for key, value in kwargs.items()}
/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/val_metrics.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_hat = torch.tensor(w_hat, dtype=torch.float32)
/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/val_metrics.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTenso

In [18]:
result_metrics[result_metrics.epoch == 99]

,epoch,train_loss,val_metric,test_regret,loss_name,hyperparameters
99,99,4.306511,0.183139,0.044372,SPO+,{}
199,99,2.063001,0.473629,0.236546,MSE,{}
299,99,0.022790,0.191735,0.029355,cosine,None


### Input SPO+ as initialization into PG Loss

In [24]:
SPO_trained = trained_models['SPO+_{}']

In [25]:
PG_result_metrics, PG_trained_models = lossfn_experiment_pipeline(X_train=generated_data['feat'],
                true_cost_train=generated_data['cost'],
                X_test=generated_data_test['feat'],
                true_cost_test=generated_data_test['cost_true'], 
                predmodel=SPO_trained,
                optmodel=optmodel,
                val_split_params={'test_size':200, 'random_state':42},
                loss_names=['PG'],          
                loss_configs={'PG': {'h':[num_data**-.125, num_data**-.25, num_data**-.5, num_data**-1], 
                                     'finite_diff_type': ['B', 'C', 'F']
                                    }
                             },                
                training_configs={'num_epochs':100,
                                  'lr': 0.01,
                                 'dataloader_params': {'batch_size':200, 'shuffle':True}},
                save_models=False
                )

/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/train.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = {key: torch.tensor(value, dtype=torch.float32) for key, value in kwargs.items()}
/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/val_metrics.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_hat = torch.tensor(w_hat, dtype=torch.float32)
/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/val_metrics.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTenso

## Combine Results

In [36]:
final_metrics = pd.concat([result_metrics, PG_result_metrics], ignore_index=True)

In [40]:
final_metrics.loc[final_metrics.groupby('loss_name')['val_metric'].idxmin()].sort_values(by='test_regret')

,epoch,train_loss,val_metric,test_regret,loss_name,hyperparameters
716,16,14.385149,0.160507,0.019160,PG,"{'h': 0.26591479484724945, 'finite_diff_type':..."
263,63,0.027388,0.145397,0.035368,cosine,None
99,99,4.306511,0.183139,0.044372,SPO+,{}
197,97,2.106824,0.444043,0.238608,MSE,{}
